# Purpose

Find a way to get data from Plugshare.com since they're not responding to my API access request. The comments and metadata from stations across different networks should be extremely useful in diagnosing electrical and non-electrical customer experience issues.

# Imports

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from rich import print
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests

from evlens.data.plugshare import Scraper

from dotenv import load_dotenv
load_dotenv(override=True)

from evlens.logs import setup_logger
logger = setup_logger("Notebook-0.1")
logger.info("TEST!")

2024-06-11_T18_44_44EDT: INFO (Notebook-0.1:L18) - TEST!


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Testing our custom scraper

In [3]:
# Electrify America in Springfield, VA mall parking lot
TEST_LOCATION = 252784

In [21]:
# Why is it taking so long to even *start* trying to exit the login dialog?!
s = Scraper(timeout=3)

# Scrape only one location that I can test via browser
df = s.scrape_plugshare_locations(TEST_LOCATION, TEST_LOCATION)
df.info()
df.head()

2024-06-11_T21_45_43EDT: INFO (evlens.data.plugshare:L169) - Beginning scraping!
Parsing stations:   0%|          | 0/1 [01:22<?, ?it/s]


RuntimeError: Unknown error trying to exit login dialog: Message: element click intercepted: Element <button class="close md-icon-button md-button md-ink-ripple" type="button" ng-transclude="" ng-click="maps.closeAuth()" aria-label="cancel" ng-show="maps.visits <= 5 || maps.gmaps" ng-disabled="maps.auth.processing" aria-hidden="false">...</button> is not clickable at point (1049, 106). Other element would receive the click: <div id="global-consent-tool-wrapper" tabindex="-1" aria-hidden="false" role="presentation" style="position: fixed; width: 100%; height: 100%; top: 0px; left: 0px; z-index: 2147483647; background: rgba(0, 0, 0, 0.5);">...</div>
  (Session info: chrome=126.0.6478.56)
Stacktrace:
0   chromedriver                        0x0000000104a97a1c chromedriver + 4389404
1   chromedriver                        0x0000000104a90328 chromedriver + 4358952
2   chromedriver                        0x00000001046acafc chromedriver + 281340
3   chromedriver                        0x000000

In [22]:
import requests

r = requests.get("https://www.plugshare.com/scripts/vendor-2cdcca5031.js")
r

<Response [200]>

In [26]:
len(r.text)

1175150

In [19]:
df.loc[0, 'Comments']

'J-1772 1 Plug 7 kW\n1 Station\nElectrify America, CCS/SAE 3 Plugs 350 kW\n3 Stations\nElectrify America, , Jun 7, 2024\ncheck_circleJ C\nBMW iX 2024, Jun 5, 2024\ncheck_circleKJva\nMercedes EQE 350350 Kilowatts, May 30, 2024\ncheck_circleKJva\nMercedes EQE 350350 Kilowatts, May 24, 2024\ncheck_circleBEV\nChevrolet Blazer EV 2024\nSo far so good! Screen is cracked on the charger, difficult to read the prompts… but I guess it doesn’t matter if it charges!, May 24, 2024\ncheck_circleKMac\nKia EV6 2022235 Kilowatts\nScreen on 1 not working but can charge through the app, 0.08 mi\n9.3 Springfield Town Center - Target - East Lot (2)\nJ-1772, 0.18 mi\n4.8 Springfield Town Center - LA Fitness\nJ-1772, 0.22 mi\n1 Springfield Town Center - Frontier Garage\nJ-1772, 0.22 mi\n10 Springfield Town Center - Target - West Lot\nNACS (Tesla), 0.23 mi\n4.7 Springfield Town Center - Maggianos\nJ-1772, , , '